In [3]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
    
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None
exp_path = Path.cwd().resolve() # path to the experiment folder

print(f"Repo Path: {repo_path}")
print(f"Experiment Path: {exp_path}")

#Libraries
import yaml
import math
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.transforms import (
    Compose,
    Resize,
    CenterCrop,
    ToTensor,
    Normalize,
    InterpolationMode,
)
import wandb
import datasets, diffusers
from diffusers import (
    UNet2DModel,
    DDPMScheduler,
)   
from diffusers import DDPMPipeline, AutoencoderKL, DiffusionPipeline
from diffusers.optimization import get_scheduler
from diffusers.utils import check_min_version
from diffusers.utils.import_utils import is_xformers_available
from transformers import CLIPTokenizer, CLIPTextModel
import logging
from accelerate.logging import get_logger
from accelerate import Accelerator

# extra
from packaging import version

# import the MRIDataset class from the dataset folder
from dataset.build_dataset import MRIDataset

/home/benet/tfg/experiments
/home/benet/tfg
Repo Path: /home/benet/tfg
Experiment Path: /home/benet/tfg/experiments/latent_finetuning


In [11]:
config_path = exp_path / 'config_latent_finetuning.yaml' # configuration file path (beter to call it from the args parser)
with open(config_path) as file: # expects the config file to be in the same directory
    config = yaml.load(file, Loader=yaml.FullLoader)

# Load the diffusion model
ldm = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
model = ldm.unet.to("cpu")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [16]:


if config['model']['prompt'] is not None:
    # Encode prompt properly
    prompt = [config['model']['prompt']] * config['processing']['batch_size']
else:
    # Make sure the text embeddings are None but in the format (batch_size, num_tokens, hidden_size)
    prompt = [""] * config['processing']['batch_size']
    
tokenizer = ldm.tokenizer
text_encoder = ldm.text_encoder.to("cpu")

# Load tokenizer and text encoder and encode prompt
text_inputs = tokenizer(prompt, padding="max_length", max_length=77, return_tensors="pt").to("cpu")
text_embeddings = text_encoder(**text_inputs).last_hidden_state

# Now you can check the shape
print(text_embeddings.shape)



torch.Size([32, 77, 768])


In [10]:
text_embeddings.shape

AttributeError: 'list' object has no attribute 'shape'